In [106]:
import pandas as pd
import glob, os, json, csv


json_dir = 'nytimes.com/'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
df = pd.concat(dfs)

df = df.dropna(subset =['title', 'maintext']) # remove entries where the title or maintext are missing.
df = df.drop(columns=['date_download','date_modify','date_publish','authors','filename','image_url','localpath','title_rss','source_domain','url','site','title_page', 'language', 'description']) # drop unused columns
df.drop_duplicates('maintext') # drop duplicate articles

# Here I spent time trying to get rid of carriage returns which were causing issues when parsing the CSV later on.
# The thing that FINALLY got rid of them was doing .split() then .join().
df['maintext'] = df['maintext'].str.replace('/n',' ')
df['maintext'] = df['maintext'].str.replace('~',' ')
df['title'] = df['title'].str.replace('/n',' ')
df['title'] = df['title'].str.replace('~',' ')
df['maintext'] = df['maintext'].str.replace('/r',' ')
df['title'] = df['title'].str.replace('/r',' ')
df['maintext'] = df['maintext'].str.strip()
df['title'] = df['title'].str.strip()
df['maintext'] = df['maintext'].str.split()
df['title'] = df['title'].str.split()
df['maintext'] = df['maintext'].str.join(sep=' ')
df['title'] = df['title'].str.join(sep=' ')
df.to_csv('nyt_all.csv', sep='~') # Create csv of all entries

In [109]:
df_train = df.iloc[:6000] # assign some entries to train
df_val = df.iloc[6001:9001]  # ' ' to validation

# create train and val CSV, delineated by ~ because that did not appear in my dataset, and I wanted to reduce any confusion when parsing.
df_train.to_csv('nyt_train.csv', sep='~', encoding='utf-8')
df_val.to_csv('nyt_val.csv', sep='~', encoding='utf-8')

In [108]:
# optional test cases
df_test = df.iloc[9002:10002]
df_test.to_csv('nyt_test.csv', sep='~', encoding='utf-8')